In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
######################## langchain. RECURSIVE ########################

In [ ]:
!pip install langchain

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [ ]:
# chunk size is measured: by number of characters.

text = "" #if you want to try one example befor iterating all examples
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex=True
)

docs = text_splitter.create_documents([text])

In [ ]:
# All outputs have been cleared to maintain the confidentiality of ShARe-13 and ShARe-14.
print(docs[0].page_content)

In [ ]:
for doc in docs:
  print(doc.page_content)

In [ ]:
# Iterate the Recursive splitter through all texts:

import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

df = pd.read_csv("/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/ShaRe14/Train-Doc-ShaRe-14.csv")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=0,
    separators=[
        "\n\n", # removed the " " separator, cuz it will allow splitting potential spans!
        "\n",
        ". ", "] "],
    is_separator_regex=False
)

rows = []

# Iterate over each row
for index, row in df.iterrows():
    text = row["Text"]
    # Perform text splitting
    docs = text_splitter.create_documents([text])

    # Iterate over each chunk and its index
    for i, doc in enumerate(docs):
        rows.append({"Index": index, "Chunk": doc.page_content})

df_output = pd.DataFrame(rows)
df_output.to_csv("output_chunks.csv", index=False)
# Now, skip the embedding-based chunking code blocks and proceed to (merging the spans with the chunks).

In [ ]:
######################## langchain. EMBEDDING-BASED CHUNCKING ########################

In [ ]:
#Semantic Chunker using OpenAI Embeddings
#Code is adapted from: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/

In [ ]:
!pip install --quiet langchain_experimental
!pip install --quiet langchain_experimental langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.7 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "--"

In [ ]:
text = "" #if you want to try one example, put it here

In [ ]:
text_splitter = SemanticChunker(OpenAIEmbeddings(),breakpoint_threshold_type="percentile",
                                breakpoint_threshold_amount=95) # here can add: breakpoint_threshold_type="percentile","standard_deviation","interquartile"
docs = text_splitter.create_documents([text])
print(docs[0].page_content)

In [ ]:
print(len(docs)) # to know how many chuncks are there

4


In [ ]:
counter = 0
for doc in docs:
  print(counter)
  print(doc.page_content)
  counter+=1

In [ ]:
### LangChain Semantic Splitter iterates through all text
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/ShaRe14/Train-Doc-ShaRe-14.csv")
rows = []

# Iterate over each row
for index, row in df.iterrows():
    text = row["Text"]
    # Perform text splitting
    docs = text_splitter.create_documents([text])

    # Iterate over each chunk and its index
    for i, doc in enumerate(docs):
        rows.append({"Index": index, "Chunk": doc.page_content})

df_output = pd.DataFrame(rows)
df_output.to_csv("trainoutput_chunks.csv", index=False)

In [ ]:
########### merging the spans with the chunks.#################

In [ ]:
#  Removing null chunks
import pandas as pd
df = pd.read_csv("/content/trainoutput_chunks.csv")

df_filtered = df[df['Chunk'].notnull()]
df_filtered.to_csv("/content/trainoutput_chunks.csv", index=False)

In [ ]:
# Now, we merge the spans with the chunks
import pandas as pd

# take the result of chunking output
file1 = pd.read_csv("/content/trainoutput_chunks.csv")
# this file2 is the resulting file of DocDiscNER_Preprocessing.ipynb. We use it here as the reference for spans.
file2 = pd.read_csv("/content/drive/MyDrive/DocDiscNER/Preprocessed_Datasets/ShaRe14/Train-Doc-ShaRe-14.csv")

# Function to calculate offsets for each chunk
def calculate_offsets(chunks):
    offsets = []
    start_offset = 0
    for chunk in chunks:
        end_offset = start_offset + len(chunk) - 1
        offsets.append(f"{start_offset}-{end_offset}")
        start_offset = end_offset + 2  # Add 2 for the newline characters
    return offsets

file1['ChunkOffsets'] = file1.groupby('Index')['Chunk'].transform(calculate_offsets)

matched_chunks = {}

for index, row in file2.iterrows():
    # Get the index value for the document
    doc_index = index

    spans = row['Spans'].split('\n')  # Split the spans by newline
    offsets = row['Offsets'].split('\n')  # Convert offsets from string to list

    # Filter chunks from file1 that have the same index as the current document
    chunks = file1[file1['Index'] == doc_index]['Chunk'].tolist()
    chunk_offsets = file1[file1['Index'] == doc_index]['ChunkOffsets'].tolist()

    # Iterate through file1 to find the correct chunk
    for chunk, chunk_offset in zip(chunks, chunk_offsets):
        chunk_spans = []
        chunk_start, chunk_end = map(int, chunk_offset.split('-'))  # Get chunk start and end
        for span, offset in zip(spans, offsets):
            start = offset.split('-', 1)[0] # Extract start offsets
            start = int(start)
            # Check if the span falls within this chunk
            if start>= chunk_start and start <= chunk_end:
                chunk_spans.append(span)
                #break  # Move to the next span

        if chunk_spans:
           matched_chunks[chunk] = chunk_spans

# Create a DataFrame from the matched_chunks dictionary
#df_matched_chunks = pd.DataFrame({'Text': list(matched_chunks.keys()), 'Spans': list(matched_chunks.values())})
df_matched_chunks = pd.DataFrame({'Text': list(matched_chunks.keys()), 'Spans': ['; '.join(value) for value in matched_chunks.values()]})

df_matched_chunks.to_csv("matched_chunks.csv", index=False)


In [ ]:
# Now, we preprocess the resulting chunks into the DocDiscNER text and spans representation.
# After chunking ShARe-13, we can optionally clean the text by removing excess details like date/time.
# Additionally, format spans with the "disorder:---" prefix.

import pandas as pd
import re

df = pd.read_csv("/content/matched_chunks.csv")

#  remove time and date expressions and symbols
def remove_time_date_symbols(text):
    # Remove time and date expressions
    text = re.sub(r"\[?\*\*\d{4}-\d{2}-\d{2}\*\*\]?\s?(at)?\s?\d{2}:\d{2}(AM|PM)?", "", text)
    pattern = re.compile(r'\[\*\*.*?\*\*\]')
    # Replace all occurrences of the pattern with an empty string
    text = re.sub(pattern, '', text)
    # Remove symbols
    text = text.replace("___________________", "").replace("||||     ||||", "")
    return text

# Apply preprocessing to the "Texts" column
df["Text"] = df["Text"].apply(remove_time_date_symbols)
df["Text"] = df["Text"].apply(lambda x: re.sub(r'\s+', ' ', x))

# format spans
def format_spans(row):
    spans = row["Spans"].split(";")
    formatted_spans = "; ".join([f"disorder: {span.strip()}" for span in spans])
    return formatted_spans

# Apply formatting to the "Spans" column
df["Spans"] = df.apply(format_spans, axis=1)

# Create "Text" and "Spans" columns
new_df = df[["Text", "Spans"]]

new_df.to_csv("Train-Doc-ShaRe-14.csv", index=False)

In [ ]:
# Optionall, if you want to print the number of tokens in each text for verification.

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Function to count tokens in a text
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/Train-Doc-ShaRe-14.csv')

# Calculate the number of tokens in the 'Text' column and create a new 'TextLength' column
df['TextLength'] = df['Text'].apply(lambda x: count_tokens(str(x)))

# Save the DataFrame with the new column to a new CSV file
df.to_csv('Train14_length.csv', index=False)

print("Number of tokens in each text have been calculated and saved to 'your_dataset_with_length.csv'")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of tokens in each text have been calculated and saved to 'your_dataset_with_length.csv'
